In [ ]:
# 🕸️ Graph Neural Networks for Fraud Detection

## 🎯 Overview

This notebook demonstrates the implementation and usage of **Graph Neural Networks (GNNs)** for advanced fraud detection. GNNs are particularly powerful for detecting **fraud rings**, **collusive behavior**, and **suspicious network patterns** that traditional machine learning methods might miss.

### 🔍 What You'll Learn

1. **📊 Graph Construction**: How to build transaction graphs from financial data
2. **🧠 GNN Architectures**: GraphSAGE, Graph Attention Networks, and Temporal GNNs  
3. **🕸️ Community Detection**: Finding fraud rings and suspicious communities
4. **🎯 Node Classification**: Predicting fraudulent entities in the network
5. **📈 Graph Features**: Extracting graph-based features for traditional models

### 🌟 Key Benefits

- **🔗 Relationship Modeling**: Capture complex relationships between entities
- **🎯 Fraud Ring Detection**: Identify coordinated fraudulent activities  
- **📊 Network Analysis**: Analyze transaction flows and patterns
- **🚀 Scalable**: Handle large-scale financial networks efficiently

---

## 🏗️ Architecture Overview

```
📊 Transaction Data
        ↓
🕸️ Graph Construction (Customers ↔ Transactions ↔ Devices)
        ↓
🧠 GNN Processing (GraphSAGE + Attention)
        ↓
🎯 Fraud Prediction + 🏘️ Community Detection
        ↓
📈 Business Actions (Block, Review, Allow)
```


In [ ]:
# Import necessary libraries
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
sys.path.append(os.path.abspath('..'))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Configuration
plt.style.use('default')
sns.set_palette("husl")

print("🕸️ Graph Neural Networks for Fraud Detection")
print("=" * 50)
print("✅ All libraries imported successfully!")
print(f"📅 Notebook initialized at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


In [ ]:
## 📊 Step 1: Data Preparation and Graph Construction

We'll start by importing our advanced GNN modules and preparing synthetic data to demonstrate graph-based fraud detection.


In [ ]:
# Import our advanced GNN modules
try:
    from src.features.graph_neural_networks import (
        TransactionGraphBuilder,
        GraphSAGEFraudDetector,
        CommunityFraudDetector,
        GraphFraudFeatureExtractor,
        GraphNeuralNetworkManager
    )
    print("✅ GNN modules imported successfully!")
except ImportError as e:
    print(f"⚠️ Import error: {e}")
    print("Note: Make sure all dependencies are installed (torch, torch-geometric, networkx)")

# Generate synthetic transaction data for demonstration
def generate_synthetic_graph_data(n_customers=1000, n_transactions=5000, fraud_rate=0.05):
    """Generate synthetic transaction data with graph structure"""
    
    np.random.seed(42)
    
    # Generate customers
    customers = [f"CUST_{i:06d}" for i in range(n_customers)]
    
    # Generate transactions with network patterns
    transactions = []
    
    # Normal transactions
    for i in range(int(n_transactions * (1 - fraud_rate))):
        customer = np.random.choice(customers)
        beneficiary = np.random.choice(customers)
        
        # Avoid self-transactions
        while beneficiary == customer:
            beneficiary = np.random.choice(customers)
        
        transaction = {
            'transaction_id': f"TXN_{i:08d}",
            'customer_id': customer,
            'beneficiary_id': beneficiary,
            'amount': np.random.lognormal(mean=4, sigma=1.5),
            'product_type': np.random.choice(['PIX', 'TED', 'DOC', 'CREDIT_CARD']),
            'timestamp': datetime.now() - timedelta(
                days=np.random.randint(0, 90),
                hours=np.random.randint(0, 24),
                minutes=np.random.randint(0, 60)
            ),
            'device_id': f"DEV_{np.random.randint(1, 10000):06d}",
            'is_fraud': 0
        }
        transactions.append(transaction)
    
    # Generate fraud ring (highly connected subgraph)
    fraud_ring_size = 20
    fraud_customers = np.random.choice(customers, fraud_ring_size, replace=False)
    
    for i in range(int(n_transactions * fraud_rate)):
        # Higher probability of transactions within fraud ring
        if np.random.random() < 0.7:
            customer = np.random.choice(fraud_customers)
            beneficiary = np.random.choice(fraud_customers)
        else:
            customer = np.random.choice(customers)
            beneficiary = np.random.choice(customers)
            
        while beneficiary == customer:
            beneficiary = np.random.choice(customers)
        
        transaction = {
            'transaction_id': f"FRAUD_{i:08d}",
            'customer_id': customer,
            'beneficiary_id': beneficiary,
            'amount': np.random.lognormal(mean=6, sigma=2),  # Higher amounts
            'product_type': np.random.choice(['PIX', 'TED'], p=[0.8, 0.2]),  # Prefer PIX
            'timestamp': datetime.now() - timedelta(
                days=np.random.randint(0, 30),  # More recent
                hours=np.random.randint(20, 24)  # Late hours
            ),
            'device_id': f"DEV_{np.random.randint(9000, 10000):06d}",  # Shared devices
            'is_fraud': 1
        }
        transactions.append(transaction)
    
    df = pd.DataFrame(transactions)
    
    print(f"📊 Generated {len(df)} transactions")
    print(f"👥 {len(df['customer_id'].unique())} unique customers")
    print(f"🔗 {len(df['beneficiary_id'].unique())} unique beneficiaries")
    print(f"⚠️ {df['is_fraud'].sum()} fraudulent transactions ({df['is_fraud'].mean():.2%})")
    
    return df

# Generate data
transactions_df = generate_synthetic_graph_data()
print("\n✅ Synthetic graph data generated!")
